# Crunchbase Raw Data Cleaning Pipeline

In [2]:
import pandas as pd

In [3]:
# manual data input 
countries = ['Peru', 'Mexico', 'Argentina', 'Brazil', 'Chile', 'Colombia', 
             'Guatemala', 'Panama', 'Ecuador', 'Uruguay', 'PuertoRico',
             #'France', 'Germany', 'Latvia',
             'Swiss', 'UK', #'Netherlands',
             'Sweden', 'Australia', 'Luxembourg', 'Ireland',
             'Poland', 'Bulgaria', 'Estonia', 'Croatia', 
             'Slovenia', 'B&H', 'Slovakia', 
             'Iran', 'Cyprus', 'Lebanon', 'SaudiArabia',
             'UAE', 'Israel', 'Qatar',
             'Malaysia', 'Indonesia', 'Thailand', 'Vietnam', 'India',
             'Morocco', 'Egypt',  'SouthAfrica', #'Madagascar',
             'Italy', 'Greece', 'Spain',
             'Japan', 'Korea', 'Taiwan', 'Kazakhstan',
             #'US', 'China',
             'Canada']

first_n = list(range(1,100,50))
last_n = list(range(50,101,50))
top_n = ['1-5', '51-100']

others = ['Netherlands_1-25', 'Masgascar_1-40']

## step 1: converting txt to csv

In [4]:
def cb_txt_to_csv(filename, i):
    
    text = ''
    
    with open(f'../data/crunchbase-raw/{filename}.txt', 'r') as f:
    
        # log row counts
        count = i

        # log previous lime
        last_line = ''

        for line in f:

            # meets index, switch to next line
            if (line == f'{count}.\n') or (line == f'{count}.'):
                text += '\n'
                last_line = line

#             # logo line
#             elif last_line == f'{count}.\n':
#                 count += 1
#                 pass

#             # logo line
#             elif 'Logo' in line:
#                 last_line = line
#                 pass

            # logo for company
            elif (last_line == f'{count}.\n') and ('Logo' in line):
                count += 1
                pass

            # logo for investors etc.
            elif 'Logo' in line:
                last_line = line
                pass

            # empty lines
            elif line == '\n':
                last_line = line
                pass

            # line with commas
            elif ',' in line:
                line = line.strip('\n').replace(',', ';') #note
                text += line
                text += ','
                last_line = line

            # data on same row, add comma 
            else:
                text += line.strip('\n')
                text += ','
                last_line = line

    with open(f'../data/crunchbase-csv/{filename}.csv', 'w') as f:
        f.write(text)

In [4]:
# note
# this is not the best method, 
# even though it worked well for a list of values in a single column;
# e.g. sentences, dates, or money values also had their commas replaced
# might need to clean the data again before using it

### run script

In [6]:
for country in countries:
    for n in top_n:
        filename = f'{country}_{n}'
        if n == '1-50':
            cb_txt_to_csv(filename, 1)
        if n == '51-100':
            cb_txt_to_csv(filename, 51)

for filename in others:
    cb_txt_to_csv(filename, 1)

In [11]:
# check
for country in countries:
    for i in range(len(first_n)):
        filename = f'{country}_{first_n[i]}-{last_n[i]}'
        try:
            df = pd.read_csv(f'../data/crunchbase-csv/{filename}.csv')
            # check correct format
            if (df.shape[0]!=50) or (df.shape[1]!=108):
                if (df.shape[1]!=106):
                    print(f'{filename}: {df.shape}')
        except:
            print(f'{filename}') #: {df.shape}')

Peru_1-50: (1, 108)
Mexico_1-50
Brazil_1-50
Guatemala_1-50
Guatemala_51-100
Panama_1-50
Panama_51-100
Ecuador_51-100
PuertoRico_1-50
PuertoRico_51-100
Swiss_51-100
UK_1-50
UK_51-100
Sweden_51-100
Luxembourg_51-100
Poland_51-100
Croatia_51-100
B&H_1-50
B&H_51-100
Slovakia_1-50
Iran_1-50
Iran_51-100
Lebanon_51-100
SaudiArabia_1-50
SaudiArabia_51-100
UAE_1-50
Qatar_1-50
Qatar_51-100
Malaysia_1-50
Malaysia_51-100
Indonesia_51-100
Thailand_51-100
Vietnam_1-50
Vietnam_51-100
Morocco_1-50
Morocco_51-100
SouthAfrica_1-50
SouthAfrica_51-100
Italy_1-50
Italy_51-100
Greece_1-50
Japan_51-100
Kazakhstan_1-50
Kazakhstan_51-100
Canada_1-50


In [ ]:
#             if (df.shape[0]!=50) or (df.shape[1]!=108):
#                 if (df.shape[1]!=106):
#                     print(f'{filename}: {df.shape}')
#         except:
#             print(f'{filename}: {df.shape}')

## step 2.1: select useful columns

In [7]:
# list(df.columns)

In [8]:
selected_cols = ['Organization Name', 'CB Rank (Company)', 
                 'Headquarters Location', 'Headquarters Regions', 
                 'Industries', 'Industry Groups', 
                 'Founders', 'Founded Date', 'Number of Employees',
                 'Estimated Revenue Range', 'Funding Status',
                 'IPO Status', 'Number of Funding Rounds', 
                 'Total Funding Amount', 'Total Equity Funding Amount',
                 'Last Funding Date', 'Last Funding Type',
                 'Top 5 Investors', 'Number of Lead Investors', 'Number of Investors',
                 #'Trend Score (7 Days)', 'Trend Score (30 Days)', 'Trend Score (90 Days)',
                 #'Average Visits (6 months)', 'Visit Duration', 'Visit Duration Growth', 'Global Traffic Rank',
                 #'Active Tech Count', 'Total Products Active'
                ]

In [9]:
# df[selected_cols].head(3)

## step 2.2: clean data and convert data types

In [10]:
df[selected_cols].dtypes

Organization Name              object
CB Rank (Company)              object
Headquarters Location          object
Headquarters Regions           object
Industries                     object
Industry Groups                object
Founders                       object
Founded Date                   object
Number of Employees            object
Estimated Revenue Range        object
Funding Status                 object
IPO Status                     object
Number of Funding Rounds        int64
Total Funding Amount           object
Total Equity Funding Amount    object
Last Funding Date              object
Last Funding Type              object
Top 5 Investors                object
Number of Lead Investors       object
Number of Investors            object
dtype: object

In [11]:
# # convert to numeric
# df['CB Rank (Company)'] = df['CB Rank (Company)'].str.replace(';','')
# df['CB Rank (Company)'] = df['CB Rank (Company)'].astype(int)

In [12]:
# # only keep country name
# df['Headquarters Location'] = df['Headquarters Location'].str.split(';').str[-1]

In [13]:
# # only keep year
# df['Founded Date'] = df['Founded Date'].str[-4:]#.split(';').str[-1]

In [14]:
# for money need to consider conversions!!
# df['Total Funding Amount']

In [15]:
def clean_cb_data(df):
    
    # reduce data
    df = df[selected_cols]
    
    # clean data
    df['CB Rank (Company)'] = df['CB Rank (Company)'].str.replace(';','')
    df['CB Rank (Company)'] = df['CB Rank (Company)'].astype(int)
    df['Headquarters Location'] = df['Headquarters Location'].str.split(';').str[-1]
    df['Founded Date'] = df['Founded Date'].str[-4:]
    
    return df

### run script

## step 3: aggregate data

In [18]:
# check
filename = 'Taiwan_top50'
df = pd.read_csv(f'../data/crunchbase-csv/{filename}.csv')
df.shape

(50, 108)

In [24]:
#seed --> series A, B, C, D --> exit (either IPO or get acquisition)
df['Industry Groups']

0       Food and Beverage; Other; Professional Services
1                Energy; Sustainability; Transportation
2                Education; Internet Services; Software
3     Biotechnology; Health Care; Science and Engine...
4         Consumer Electronics; Hardware; Manufacturing
5     Artificial Intelligence; Data and Analytics; I...
6     Consumer Electronics; Hardware; Information Te...
7     Apps; Content and Publishing; Internet Service...
8     Commerce and Shopping; Community and Lifestyle...
9     Apps; Internet Services; Media and Entertainme...
10    Consumer Electronics; Energy; Hardware; Manufa...
11    Apps; Consumer Goods; Media and Entertainment;...
12    Biotechnology; Health Care; Science and Engine...
13                                   Other; Real Estate
14    Consumer Electronics; Hardware; Manufacturing;...
15    Advertising; Data and Analytics; Mobile; Other...
16                               Apps; Gaming; Software
17    Consumer Electronics; Hardware; Informatio